## Analyze A/B Test Results

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)
- [Conclusions](#conclusions)
- [Resources](#Resources)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [2]:
# read data
df = pd.read_csv('ab_data.csv')

FileNotFoundError: [Errno 2] File b'ab_data.csv' does not exist: b'ab_data.csv'

b. Use the below cell to find the number of rows in the dataset.

In [ ]:
# check rows with shape method
df.shape

In [ ]:
# info helps to get an overview as well
df.info()

c. The number of unique users in the dataset.

In [ ]:
# use unique method
users_unique = df.user_id.nunique()
users_unique

d. The proportion of users converted.

In [ ]:
# proportions of user who converted (i.e. converted = 1)
df[df.converted == 1].user_id.count()/df.shape[0]

# alternatively
df.converted.mean()

e. The number of times the `new_page` and `treatment` don't line up.

In [ ]:
# there are two dimentions to this 
# 1. get the number where landing_page equals new_page but group is not treatment
# 2. get the number where landing_page not new_page but group is treatment
# adding these two will give total misalignment
df[(df.landing_page == 'new_page') & (df.group != 'treatment')].user_id.count() + df[(df.landing_page != 'new_page') & (df.group == 'treatment')].user_id.count()

f. Do any of the rows have missing values?

In [ ]:
# we find the rows with missing values with isnull
df.isnull().sum()

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
# using the same logic from e.
df2 = df.drop(df[((df.landing_page == 'new_page') & (df.group != 'treatment')) | ((df.landing_page != 'new_page') & (df.group == 'treatment'))].index)

In [ ]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [ ]:
# check for info
df2.info()

In [ ]:
df2.user_id.nunique()

`df.info()` shows 290585 entries but `df2.user_id.nunique()` gives 290584. Meaning there is one duplicate.

b. There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
# here we look for duplicated user_id, but we show both of those in our result by keep=False
df2.user_id[df2.user_id.duplicated(keep=False)]

c. What is the row information for the repeat **user_id**? 

In [ ]:
# at what index duplicated user_id is
df2[df2.user_id.duplicated(keep=False)]

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [ ]:
# we can remove duplicates in multiple ways
# removing by index, removing by first or last entry as well as by unique column timestamp
df2 = df2[df2.timestamp != '2017-01-09 05:37:58.781806']

In [ ]:
# quick sanity check
df2.user_id.nunique()

`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
df2.converted.mean()

# alternatively
df2.describe().loc['mean'].converted

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
# logic: query the converted column where group column value is control and take mean
df2.converted[df2.group == 'control'].mean()

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
# same logic as above, here we do it for treatment instead of control
df2.converted[df2.group == 'treatment'].mean()

d. What is the probability that an individual received the new page?

In [ ]:
df2[df2.landing_page == 'new_page'].count()/df2.shape[0]

e. Use the results in the previous two portions of this question to suggest if you think there is evidence that one page leads to more conversions?  Write your response below.


- **Given the probability that an individual received the new page is 0.50006, suggests that there is an equal chance that an individual received the old page at the same time. **
- **Besides, the probability of conversion from control group and treatment group is almost similar. Just different at 3rd decimal point.**
- **Within such circumstances we can not confidently say that one page leads to more conversions. In fact, it is quite the opposite.**

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

**Null Hypothesis: ** The probability of old page better than new page is greater or equal to probability of new page. 

**Alternative Hypothesis: ** The probability of new page being better than new page is less than that of a new page.

$$ H_{0} : p_{old} - p_{new} \geq 0$$
$$ H_{A} : p_{old} - p_{new} < 0$$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

a. What is the **convert rate** for $p_{new}$ under the null? 

In [ ]:
# convert rate is how many users converted (i.e. converted = 1)
p_new = df2.converted.mean()
p_new

b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [ ]:
p_old = df2.converted.mean() # the same!!!
p_old

c. What is $n_{new}$?

In [ ]:
# number of users who got the new page (i.e. group = treatment)
n_new = df2[df2.group == 'treatment'].count()[0]
n_new

d. What is $n_{old}$?

In [ ]:
# number of users who reamined with old page (i.e. group = control)
n_old = df2[df2.group == 'control'].count()[0]
n_old

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

**NOTE: **
> - Below few cells will see alternative method, please run both of those in two different runs. They both give different results! <br>
> - As a matter of fact they **should not.** <br>
> - The difference will be visible in the histogram and in p-value

In [ ]:
# simulating n_new transaction that have probabiliy of receiving new page, using binomial gives the outout as the number of 1's
new_page_converted = np.random.binomial(1,p_new,n_new) 
new_page_converted.mean()

In [ ]:
# alternatively
new_converted_simulation = np.random.binomial(n_new, p_new, 10000)/n_new
new_converted_simulation.mean()

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
# simulating n_new transaction that have probabiliy of having old page, using binomial gives the outout as the number of 1's
old_page_converted = np.random.binomial(1,p_old,n_old)
old_page_converted.mean()

In [ ]:
# alternatively
old_converted_simulation = np.random.binomial(n_old, p_old, 10000)/n_old
old_converted_simulation.mean()

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
# probabilty of new and old page based upon simulated values in e. and f.
p_new = new_page_converted.mean()
p_old = old_page_converted.mean()
p_new - p_old

In [ ]:
# alternatively
new_converted_simulation.mean() - old_converted_simulation.mean()

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in **p_diffs**.

In [ ]:
p_diffs = []
for _ in range(10000):
    new_page_converted = np.random.binomial(1,p_new,n_new)  # bootstrapping
    old_page_converted = np.random.binomial(1,p_old,n_old) # bootstrapping
    p_diffs.append(new_page_converted.mean() - old_page_converted.mean())

In [ ]:
# alternatively
p_diffs_alt = new_converted_simulation - old_converted_simulation

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
# convert p_diffs to numpy array for array based computations in the future
p_diffs = np.array(p_diffs)

# histogram
plt.hist(p_diffs)
plt.grid()
plt.axvline(p_diffs.mean(), color='r', label='mean')
plt.legend();

In [ ]:
# alternatively
# histogram
plt.hist(p_diffs_alt)
plt.grid()
plt.axvline(p_diffs_alt.mean(), color='r', label='mean')
plt.legend();

As expected the mean is very close to 0. 

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
actual_diff = df2.converted[df2.group == 'treatment'].mean() - df2.converted[df2.group == 'control'].mean()
(actual_diff < p_diffs).mean()

In [ ]:
# alternatively
actual_diff = df2.converted[df2.group == 'treatment'].mean() - df2.converted[df2.group == 'control'].mean()
(actual_diff < p_diffs_alt).mean()

k. In words, explain what you just computed in part **j.**.  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

- **What we computed in part j. is called p-value in scientific studies.**
- **p-value is the probability of observing your statistic (or one more extreme in favor of the alternative) if the null hypothesis is true.**
- **In our case the p-value is so big that we can confidently say that we fail to reject null hypothesis**

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
# quick check before diving in
df2.head()

In [ ]:
import statsmodels.api as sm

convert_old = df2[df2.group == 'control'].converted.sum()
convert_new = df2[df2.group == 'treatment'].converted.sum()
n_old = df2[df2.group == 'control'].converted.count()
n_new = df2[df2.group == 'treatment'].converted.count()

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

From the link, we have all the values required. However, it is worth noticing that the argument alternative in the function is 'two-sided' by default. But our hypothesis is not designed around 'either-or' scenario at this point. Therefore, the argument alternative will be 'smaller', meaning one-tailed experiment. For further reading please refer to [wikipedia](https://en.wikipedia.org/wiki/One-_and_two-tailed_tests)  article. As well as, a great informative [video](https://www.youtube.com/watch?v=fXOS4Q3nJQY) about z-scores.

In [ ]:
# based upon the documentation
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')
z_score, p_value

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

- **A z-score represents how many standard deviations away our data point is from the mean**
- **A positive z-score suggests that our data point is on the right side of the mean line on the bell curve**
- **p-value of 0.9050 is very close to the p-value we computed earlier in part j.**
- **With this computation, we can confidently say we fail to reject null hypothesis**

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**This is a case of Logistic Regression. In this scenario, we want to predict something that has only two possible outcomes.**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a colun for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
# quick consistency check
df2.head()

In [ ]:
# copy the dataframe to a new one, for clarity
df2_copy = df2.copy()

In [ ]:
# another sanity check, if True, go ahead
df2_copy.shape == df2.shape

In [ ]:
# add an intercept
df2_copy['intercept'] = 1

# using pd.get_dummies to convert the 'landing_page' and 'group' to 1s and 0s
df2_copy[['landing_page_new', 'landing_page_old']] = pd.get_dummies(df2_copy['landing_page'])
df2_copy[['ab_page_control','ab_page_treatment']] = pd.get_dummies(df2_copy['group'])

In [ ]:
# here, we will use landing_page_old and ab_page_control as our baselines, therefore remove those
df2_copy.drop(['landing_page_old','ab_page_control'], axis=1, inplace=True)

In [ ]:
df2_copy.head()

c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

In [ ]:
logit_mod = sm.Logit(df2_copy['converted'], df2_copy[['intercept', 'ab_page_treatment']])
results = logit_mod.fit()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results.summary()

In [ ]:
# interprete the coefficients
1/np.exp(results.params[1])

In [ ]:
# additional: question posed by previous reviewer
# how to get p-value as if it were a one-tailed test
1-0.190/2

- For each 1 unit decrease in `ab_page_treatment`, conversion is 1.015 time likely holding all the other variables constant. Which is not really why we would launch a new_page!

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in the **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

- **As we already built the base for z-score and p-value in part-m, this p-value is different of course.**
- **The logistic regression is two-tailed case. We are exploring only two possible outcomes and it can go either way.**
- **Interesting part to observe is z-score, absolute of which confirms the number in part-m.**

The null and alternative hypothesis associated with regression model will be as following:
$$ H_{0}: p_{old} - p_{new} = 0$$
$$ H_{0}: p_{old} - p_{new} \neq 0$$

In [ ]:
# duration of the experiment
duration = np.array(pd.to_datetime(df.timestamp).sort_values(ascending=True))
td = duration[-1] - duration[0]
days = td.astype('timedelta64[D]')
days / np.timedelta64(1, 'D')

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**There can be many other factors that can be taken into consideration to add into our regression model.**
- **One of the first to consider would be the duration. The duration of this experiment was 21 days. Too short, it would be advisable to increase the duration**
- **Geographic location is another important factor. If the page is available in multiple languages, for example, it can diversify the sample**
- **Parameters like click through rate is another factor to consider**

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy varaibles.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
# read file and join the dfs
countries_df = pd.read_csv('./countries.csv')
df_new = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')

In [ ]:
# quick check
df_new.head()

In [ ]:
# Create the necessary dummy variables
df_new[['canada','uk','us']] = pd.get_dummies(df_new['country'])

In [ ]:
# let's consider US being our baseline, therefore, we drop US
df_new.drop(['us'], axis=1, inplace=True)

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
df_new.head()

In [ ]:
# fitting linear model
df_new['intercept'] = 1

logit_mod = sm.Logit(df_new['converted'], df_new[['intercept','canada','uk']])
results = logit_mod.fit()

In [ ]:
results.summary()

In [ ]:
1/np.exp(-0.0408), np.exp(0.0099)

Above results can be read as:
- users from uk are 1.009 times more likely to convert as compared to users from us
- users from canada are 1.042 times more likely to less convert as compared to users from us

<a id='conclusions'></a>
## Conclusions

Within the framework this project, we tried to understand whether the company should implement a new page or keep the old page with following:
- Probability based approach
- A/B test
- Regression approach

**Probability based approach:**
- Given that an individual was in the treatment group, the probability they converted is 0.118807
- Given that an individual was in the control group, the probability they converted is 0.120386
- We find that old page does better, but by a very tiny margin.
- Change aversion, test span durations and other potentially influencing factors are not accounted for. So, we cannot state with certainty that one page leads to more conversions. This is even more important due to almost similar perforamnce of both pages.

**A/B test:**
  - We simulated our user groups with respect to conversions
  - We found the p_value to be 0.8879
  - With such a p-value, we failed to reject null hypothesis
  - We find that the z-score of 1.3109 is less than the critical value of 1.6448. So, we accept the null hypothesis.
  - As regards the conversion rates of the old and new pages, we find that old pages are only minutely better than new pages.
  - These values agree with the findings in parts j. and k.
  
**Regression Approach:**
  - We looked at exploring two possible outcomes. Whether new page is better or not.
  - With logistic regression results, we again encountered same z-score as well as p-value of 0.190, corresponding two-tailed case
  - Adding geographic location of the users, we tried to find if any specific country had an impact on conversion
  - The result gave a similar outlook and suggested that the countries have no impact on the conversion rate.
  
**Consideration:**
  - These inferences are strictly based on data on hand. This analysis acknowledges its limitations due to factors not included in the data. 
  - We also looked at the duration of the experiment (21 days).This is a relatively short period of time to run the A/B test.

**References**

https://github.com/ishita-joshi/Analyze-A-B-Test-Results/blob/dc4d13cd935dd0447622f9c3ce6dcddda43d06bf/IshitaJ_Analyze_ab_test_results.ipynb